# 📊 Comprehensive Statistical Analysis of Indian Rainfall

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, shapiro

In [ ]:
#=== Task 1 Name: ===

#===============

#=== Task 2 Name: ===

#===============

#=== Task 3 Name: ===

#===============
#=== Task 4 Name: ===

#===============

In [ ]:
# ===========================
# 1) LOAD + CLEAN DATA
# ===========================
df = pd.read_csv("rainfall in india 1901-2015.csv")

# Keep only the yearly rainfall column
# (Change the column name if your file differs)
rain = df["ANNUAL RAINFALL"].dropna()

# ===========================
# 2) CALCULATE μ AND σ
# ===========================
mu = rain.mean()
sigma = rain.std()

print("Mean rainfall (μ):", mu)
print("Std deviation (σ):", sigma)

# ===========================
# 3) NORMALITY CHECK
# ===========================
stat, p_value = shapiro(rain)
print("\nShapiro-Wilk p-value:", p_value)

# ===========================
# 4) HISTOGRAM + NORMAL CURVE
# ===========================

plt.figure(figsize=(12,6))

# Histogram of rainfall
plt.hist(rain, bins=20, density=True, alpha=0.6, label="Rainfall Histogram")

# Create x-values across the data range
x = np.linspace(min(rain), max(rain), 500)

# Normal PDF curve
pdf = norm.pdf(x, mu, sigma)

plt.plot(x, pdf, linewidth=3, label="Normal Distribution PDF")

plt.title("Rainfall Distribution with Normal Fit")
plt.xlabel("Annual Rainfall (mm)")
plt.ylabel("Density")
plt.legend()
plt.grid(True)
plt.show()

# ===========================
# 5) EXAMPLE PROBABILITY QUERIES
# ===========================
# Probability of rainfall > 1100 mm
p_high = 1 - norm.cdf(1100, mu, sigma)

# Probability between 900 and 1200
p_range = norm.cdf(1200, mu, sigma) - norm.cdf(900, mu, sigma)

print("\nP(rainfall > 1100 mm):", p_high)
print("P(900 < rainfall < 1200):", p_range)
